In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import logging

logger = logging.getLogger(__name__)

In [3]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=200, n_samples=20000)
dm.setup()

In [4]:
from opensynth.models.faraday.model import FaradayVAE
# Option to pass in your own encoder architecture in the future
model = FaradayVAE(class_dim=2, latent_dim=16, learning_rate=0.001, mse_weight=3)

In [5]:
trainer = pl.Trainer(max_epochs=2, accelerator="auto")
trainer.fit(model, dm)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/shengchai/.local/share/virtualenvs/OpenSynth-EhRIPYd3/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name           | Type                    | Params | Mode 
-------------------------------------------------------------------
0 | encoder        | Encoder                 | 201 K  | train
1 | decoder        | Decoder                 | 200 K  | train
2 | reparametriser | ReparametrisationModule | 544    | 

Epoch 1: 100%|██████████| 100/100 [00:02<00:00, 35.72it/s, v_num=65, total_loss=3.420, mmd_loss=0.196, mse_loss=1.570, quantile_loss=1.650]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 100/100 [00:02<00:00, 35.31it/s, v_num=65, total_loss=3.420, mmd_loss=0.196, mse_loss=1.570, quantile_loss=1.650]


In [10]:
from opensynth.models.faraday.model import FaradayModel
import numpy as np
import torch

In [11]:
gmm_data_module = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=500, n_samples=1000, outlier_path=outlier_path)
gmm_data_module.setup()

In [12]:
gmm_model = FaradayModel(vae_module=model, n_components=10, max_iter=100, tol=1e-2)
gmm_model.train_gmm(dm=gmm_data_module)

2it [00:00, 36.81it/s]


In [15]:
gmm_samples = gmm_model.sample_gmm(10000)
gmm_samples_kwh = dm.dataset.reconstruct(gmm_samples["kwh"])
gmm_samples_kwh = torch.clip(gmm_samples_kwh, min=0)